In [45]:
import os
import numpy as np
import pandas as pd
import deepchem as dc
import gc
from tqdm import tqdm_notebook
from scipy.spatial.distance import norm
from glob import glob
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig

In [46]:
# %load ../xyz2mol.py
##
# Copied from: https://github.com/jensengroup/xyz2mol
#
# Written by Jan H. Jensen based on this paper Yeonjoon Kim and Woo Youn Kim
# "Universal Structure Conversion Method for Organic Molecules: From Atomic Connectivity
# to Three-Dimensional Geometry" Bull. Korean Chem. Soc. 2015, Vol. 36, 1769-1777 DOI: 10.1002/bkcs.10334
#
from rdkit import Chem
from rdkit.Chem import AllChem
import itertools
from rdkit.Chem import rdmolops
from collections import defaultdict
import copy
import networkx as nx #uncomment if you don't want to use "quick"/install networkx


global __ATOM_LIST__
__ATOM_LIST__ = [ x.strip() for x in ['h ','he', \
      'li','be','b ','c ','n ','o ','f ','ne', \
      'na','mg','al','si','p ','s ','cl','ar', \
      'k ','ca','sc','ti','v ','cr','mn','fe','co','ni','cu', \
      'zn','ga','ge','as','se','br','kr', \
      'rb','sr','y ','zr','nb','mo','tc','ru','rh','pd','ag', \
      'cd','in','sn','sb','te','i ','xe', \
      'cs','ba','la','ce','pr','nd','pm','sm','eu','gd','tb','dy', \
      'ho','er','tm','yb','lu','hf','ta','w ','re','os','ir','pt', \
      'au','hg','tl','pb','bi','po','at','rn', \
      'fr','ra','ac','th','pa','u ','np','pu'] ]


def get_atom(atom):
    global __ATOM_LIST__
    atom = atom.lower()
    return __ATOM_LIST__.index(atom) + 1


def getUA(maxValence_list, valence_list):
    UA = []
    DU = []
    for i, (maxValence,valence) in enumerate(zip(maxValence_list, valence_list)):
        if maxValence - valence > 0:
            UA.append(i)
            DU.append(maxValence - valence)
    return UA,DU


def get_BO(AC,UA,DU,valences,UA_pairs,quick):
    BO = AC.copy()
    DU_save = []

    while DU_save != DU:
        for i,j in UA_pairs:
            BO[i,j] += 1
            BO[j,i] += 1

        BO_valence = list(BO.sum(axis=1))
        DU_save = copy.copy(DU)
        UA, DU = getUA(valences, BO_valence)
        UA_pairs = get_UA_pairs(UA,AC,quick)[0]

    return BO


def valences_not_too_large(BO,valences):
    number_of_bonds_list = BO.sum(axis=1)
    for valence, number_of_bonds in zip(valences,number_of_bonds_list):
        if number_of_bonds > valence:
            return False

    return True


def BO_is_OK(BO,AC,charge,DU,atomic_valence_electrons,atomicNumList,charged_fragments):
    Q = 0 # total charge
    q_list = []
    if charged_fragments:
        BO_valences = list(BO.sum(axis=1))
        for i,atom in enumerate(atomicNumList):
            q = get_atomic_charge(atom,atomic_valence_electrons[atom],BO_valences[i])
            Q += q
            if atom == 6:
                number_of_single_bonds_to_C = list(BO[i,:]).count(1)
                if number_of_single_bonds_to_C == 2 and BO_valences[i] == 2:
                    Q += 1
                    q = 2
                if number_of_single_bonds_to_C == 3 and Q + 1 < charge:
                    Q += 2
                    q = 1

            if q != 0:
                q_list.append(q)

    if (BO-AC).sum() == sum(DU) and charge == Q and len(q_list) <= abs(charge):
        return True
    else:
        return False


def get_atomic_charge(atom,atomic_valence_electrons,BO_valence):
    if atom == 1:
        charge = 1 - BO_valence
    elif atom == 5:
        charge = 3 - BO_valence
    elif atom == 15 and BO_valence == 5:
        charge = 0
    elif atom == 16 and BO_valence == 6:
        charge = 0
    else:
        charge = atomic_valence_electrons - 8 + BO_valence

    return charge

def clean_charges(mol):
    # this hack should not be needed any more but is kept just in case

    rxn_smarts = ['[N+:1]=[*:2]-[C-:3]>>[N+0:1]-[*:2]=[C-0:3]',
                  '[N+:1]=[*:2]-[O-:3]>>[N+0:1]-[*:2]=[O-0:3]',
                  '[N+:1]=[*:2]-[*:3]=[*:4]-[O-:5]>>[N+0:1]-[*:2]=[*:3]-[*:4]=[O-0:5]',
                  '[#8:1]=[#6:2]([!-:6])[*:3]=[*:4][#6-:5]>>[*-:1][*:2]([*:6])=[*:3][*:4]=[*+0:5]',
                  '[O:1]=[c:2][c-:3]>>[*-:1][*:2][*+0:3]',
                  '[O:1]=[C:2][C-:3]>>[*-:1][*:2]=[*+0:3]']

    fragments = Chem.GetMolFrags(mol,asMols=True,sanitizeFrags=False)

    for i,fragment in enumerate(fragments):
        for smarts in rxn_smarts:
            patt = Chem.MolFromSmarts(smarts.split(">>")[0])
            while fragment.HasSubstructMatch(patt):
                rxn = AllChem.ReactionFromSmarts(smarts)
                ps = rxn.RunReactants((fragment,))
                fragment = ps[0][0]
        if i == 0:
            mol = fragment
        else:
            mol = Chem.CombineMols(mol,fragment)

    return mol


def BO2mol(mol,BO_matrix, atomicNumList,atomic_valence_electrons,mol_charge,charged_fragments):
    # based on code written by Paolo Toscani

    l = len(BO_matrix)
    l2 = len(atomicNumList)
    BO_valences = list(BO_matrix.sum(axis=1))

    if (l != l2):
        raise RuntimeError('sizes of adjMat ({0:d}) and atomicNumList '
            '{1:d} differ'.format(l, l2))

    rwMol = Chem.RWMol(mol)

    bondTypeDict = {
        1: Chem.BondType.SINGLE,
        2: Chem.BondType.DOUBLE,
        3: Chem.BondType.TRIPLE
    }

    for i in range(l):
        for j in range(i + 1, l):
            bo = int(round(BO_matrix[i, j]))
            if (bo == 0):
                continue
            bt = bondTypeDict.get(bo, Chem.BondType.SINGLE)
            rwMol.AddBond(i, j, bt)
    mol = rwMol.GetMol()

    if charged_fragments:
        mol = set_atomic_charges(mol,atomicNumList,atomic_valence_electrons,BO_valences,BO_matrix,mol_charge)
    else:
        mol = set_atomic_radicals(mol,atomicNumList,atomic_valence_electrons,BO_valences)

    return mol

def set_atomic_charges(mol,atomicNumList,atomic_valence_electrons,BO_valences,BO_matrix,mol_charge):
    q = 0
    for i,atom in enumerate(atomicNumList):
        a = mol.GetAtomWithIdx(i)
        charge = get_atomic_charge(atom,atomic_valence_electrons[atom],BO_valences[i])
        q += charge
        if atom == 6:
            number_of_single_bonds_to_C = list(BO_matrix[i,:]).count(1)
            if number_of_single_bonds_to_C == 2 and BO_valences[i] == 2:
                    q += 1
                    charge = 0
            if number_of_single_bonds_to_C == 3 and q + 1 < mol_charge:
                    q += 2
                    charge = 1

        if (abs(charge) > 0):
            a.SetFormalCharge(int(charge))

    # shouldn't be needed anymore bit is kept just in case
    #mol = clean_charges(mol)

    return mol


def set_atomic_radicals(mol,atomicNumList,atomic_valence_electrons,BO_valences):
    # The number of radical electrons = absolute atomic charge
    for i,atom in enumerate(atomicNumList):
        a = mol.GetAtomWithIdx(i)
        charge = get_atomic_charge(atom,atomic_valence_electrons[atom],BO_valences[i])

        if (abs(charge) > 0):
            a.SetNumRadicalElectrons(abs(int(charge)))

    return mol

def get_bonds(UA,AC):
    bonds = []

    for k,i in enumerate(UA):
        for j in UA[k+1:]:
            if AC[i,j] == 1:
                bonds.append(tuple(sorted([i,j])))

    return bonds

def get_UA_pairs(UA,AC,quick):
    bonds = get_bonds(UA,AC)
    if len(bonds) == 0:
        return [()]

    if quick:
        G=nx.Graph()
        G.add_edges_from(bonds)
        UA_pairs = [list(nx.max_weight_matching(G))]
        return UA_pairs

    max_atoms_in_combo = 0
    UA_pairs = [()]
    for combo in list(itertools.combinations(bonds, int(len(UA)/2))):
        flat_list = [item for sublist in combo for item in sublist]
        atoms_in_combo = len(set(flat_list))
        if atoms_in_combo > max_atoms_in_combo:
            max_atoms_in_combo = atoms_in_combo
            UA_pairs = [combo]
 #           if quick and max_atoms_in_combo == 2*int(len(UA)/2):
 #               return UA_pairs
        elif atoms_in_combo == max_atoms_in_combo:
            UA_pairs.append(combo)

    return UA_pairs

def AC2BO(AC,atomicNumList,charge,charged_fragments,quick):
    # TODO
    atomic_valence = defaultdict(list)
    atomic_valence[1] = [1]
    atomic_valence[6] = [4]
    atomic_valence[7] = [4,3]
    atomic_valence[8] = [2,1]
    atomic_valence[9] = [1]
    atomic_valence[14] = [4]
    atomic_valence[15] = [5,4,3]
    atomic_valence[16] = [6,4,2]
    atomic_valence[17] = [1]
    atomic_valence[32] = [4]
    atomic_valence[35] = [1]
    atomic_valence[53] = [1]


    atomic_valence_electrons = {}
    atomic_valence_electrons[1] = 1
    atomic_valence_electrons[6] = 4
    atomic_valence_electrons[7] = 5
    atomic_valence_electrons[8] = 6
    atomic_valence_electrons[9] = 7
    atomic_valence_electrons[14] = 4
    atomic_valence_electrons[15] = 5
    atomic_valence_electrons[16] = 6
    atomic_valence_electrons[17] = 7
    atomic_valence_electrons[32] = 4
    atomic_valence_electrons[35] = 7
    atomic_valence_electrons[53] = 7

    # make a list of valences, e.g. for CO: [[4],[2,1]]
    valences_list_of_lists = []
    for atomicNum in atomicNumList:
        valences_list_of_lists.append(atomic_valence[atomicNum])

    # convert [[4],[2,1]] to [[4,2],[4,1]]
    valences_list = list(itertools.product(*valences_list_of_lists))

    best_BO = AC.copy()

    # implemenation of algorithm shown in Figure 2
    # UA: unsaturated atoms
    # DU: degree of unsaturation (u matrix in Figure)
    # best_BO: Bcurr in Figure
    #

    for valences in valences_list:
        AC_valence = list(AC.sum(axis=1))
        UA,DU_from_AC = getUA(valences, AC_valence)

        if len(UA) == 0 and BO_is_OK(AC,AC,charge,DU_from_AC,atomic_valence_electrons,atomicNumList,charged_fragments):
            return AC,atomic_valence_electrons

        UA_pairs_list = get_UA_pairs(UA,AC,quick)
        for UA_pairs in UA_pairs_list:
            BO = get_BO(AC,UA,DU_from_AC,valences,UA_pairs,quick)
            if BO_is_OK(BO,AC,charge,DU_from_AC,atomic_valence_electrons,atomicNumList,charged_fragments):
                return BO,atomic_valence_electrons

            elif BO.sum() >= best_BO.sum() and valences_not_too_large(BO,valences):
                best_BO = BO.copy()

    return best_BO,atomic_valence_electrons


def AC2mol(mol,AC,atomicNumList,charge,charged_fragments,quick):
    # convert AC matrix to bond order (BO) matrix
    BO,atomic_valence_electrons = AC2BO(AC,atomicNumList,charge,charged_fragments,quick)

    # add BO connectivity and charge info to mol object
    mol = BO2mol(mol,BO, atomicNumList,atomic_valence_electrons,charge,charged_fragments)

    return mol


def get_proto_mol(atomicNumList):
    mol = Chem.MolFromSmarts("[#"+str(atomicNumList[0])+"]")
    rwMol = Chem.RWMol(mol)
    for i in range(1,len(atomicNumList)):
        a = Chem.Atom(atomicNumList[i])
        rwMol.AddAtom(a)

    mol = rwMol.GetMol()

    return mol


def get_atomicNumList(atomic_symbols):
    atomicNumList = []
    for symbol in atomic_symbols:
        atomicNumList.append(get_atom(symbol))
    return atomicNumList


def read_xyz_file(filename):

    atomic_symbols = []
    xyz_coordinates = []

    with open(filename, "r") as file:
        for line_number,line in enumerate(file):
            if line_number == 0:
                num_atoms = int(line)
            elif line_number == 1:
                if "charge=" in line:
                    charge = int(line.split("=")[1])
                else:
                    charge = 0
            else:
                atomic_symbol, x, y, z = line.split()
                atomic_symbols.append(atomic_symbol)
                xyz_coordinates.append([float(x),float(y),float(z)])

    atomicNumList = get_atomicNumList(atomic_symbols)

    return atomicNumList,charge,xyz_coordinates

def xyz2AC(atomicNumList,xyz):
    import numpy as np
    mol = get_proto_mol(atomicNumList)

    conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        conf.SetAtomPosition(i,(xyz[i][0],xyz[i][1],xyz[i][2]))
    mol.AddConformer(conf)

    dMat = Chem.Get3DDistanceMatrix(mol)
    pt = Chem.GetPeriodicTable()

    num_atoms = len(atomicNumList)
    AC = np.zeros((num_atoms,num_atoms)).astype(int)

    for i in range(num_atoms):
        a_i = mol.GetAtomWithIdx(i)
        Rcov_i = pt.GetRcovalent(a_i.GetAtomicNum())*1.30
        for j in range(i+1,num_atoms):
            a_j = mol.GetAtomWithIdx(j)
            Rcov_j = pt.GetRcovalent(a_j.GetAtomicNum())*1.30
            if dMat[i,j] <= Rcov_i + Rcov_j:
                AC[i,j] = 1
                AC[j,i] = 1

    return AC,mol,dMat

def chiral_stereo_check(mol):
    Chem.DetectBondStereochemistry(mol,-1)
    Chem.AssignStereochemistry(mol, flagPossibleStereoCenters=True, force=True)
    Chem.AssignAtomChiralTagsFromStructure(mol,-1)
    return mol

def xyz2mol(atomicNumList, charge, xyz_coordinates, charged_fragments, quick,
            check_chiral_stereo=True):

    # Get atom connectivity (AC) matrix, list of atomic numbers, molecular charge,
    # and mol object with no connectivity information
    AC,mol,dMat = xyz2AC(atomicNumList, xyz_coordinates)

    # Convert AC to bond order matrix and add connectivity and charge info to mol object
    new_mol = AC2mol(mol, AC, atomicNumList, charge, charged_fragments, quick)

    # sanitize
    try: Chem.SanitizeMol(new_mol)
    except ValueError as e: print(e)

    # Check for stereocenters and chiral centers
    if check_chiral_stereo:
        try: new_mol = chiral_stereo_check(new_mol)
        except ValueError as e: print(e)

    return new_mol,dMat

In [85]:
## Constants
# scalar coupling types
TYPES     = np.array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC',
                      '3JHN'])
TYPES_MAP = {t: i for i, t in enumerate(TYPES)}

# feature definition file
FDEF = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')

# feature collections
SYMBOLS        = ['H', 'C', 'N', 'O', 'F']
DEGREES        = [1, 2, 3, 4, 5]
HYBRIDIZATIONS = [
    Chem.rdchem.HybridizationType.SP,
    Chem.rdchem.HybridizationType.SP2,
    Chem.rdchem.HybridizationType.SP3,
    Chem.rdchem.HybridizationType.UNSPECIFIED
]

# feature maps
ATOMIC_RADIUS = {'H': 0.43, 'C': 0.82, 'N': 0.8, 'O': 0.78, 'F': 0.76} # With fudge factor
ELECTRO_NEG   = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}

# feature column names
ATOM_COLUMNS = [f'type_{a}' for a in SYMBOLS] \
             + [f'degree_{i}' for i in DEGREES] \
             + ['SP', 'SP2', 'SP3', 'hybridization_unspecified'] \
             + ['aromatic', 'formal_charge', 'atomic_num',
                'radius', 'electro_negativity', 'donor', 
                'acceptor']
EDGE_COLUMNS = ['single', 'double', 'triple', 'aromatic', 
                'conjugated', 'in_ring', 'dist', 'normed_dist']

# feature dimensions
N_EDGE_FEATURES        = 8
N_ATOM_FEATURES        = 21
MAX_N_ATOMS            = 29
MAX_N_BONDS            = 58
N_TYPES                = len(TYPES)

# paths
DATA_PATH = '../data/'
PATH      = '../tmp/'

In [72]:
## Helper functions
def array_to_csv(arr, f_name, n, fmt='%.10f'):
    "Writes numpy array 'arr' to csv file."
    f = PATH + f_name + '.csv'
    np.savetxt(f, arr.reshape(n, -1), delimiter=',', fmt=fmt)

def print_progress(i):
    if (i%10000)==0: print(i)

def clear_memory(var_strs):
    for var_str in var_strs: del globals()[var_str]
    gc.collect()

In [73]:
## import data
train_df = pd.read_csv(DATA_PATH+'train.csv')
test_df = pd.read_csv(DATA_PATH+'test.csv')
structures_df = pd.read_csv(DATA_PATH+'structures.csv')

In [74]:
## Process dataframes
def map_atom_info(df, atom_idx, struct_df):
    df = pd.merge(df, struct_df, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])

    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

def add_dist(df, struct_df):
    df = map_atom_info(df, 0, struct_df)
    df = map_atom_info(df, 1, struct_df)
    p_0 = df[['x_0', 'y_0', 'z_0']].values
    p_1 = df[['x_1', 'y_1', 'z_1']].values
    df['dist'] = np.linalg.norm(p_0 - p_1, axis=1)
    df.drop(
        columns=['atom_0', 'atom_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'],
        inplace=True
    )
    return df

def add_atom_counts(df, struct_df):
    pd.options.mode.chained_assignment = None
    atoms_per_mol_df = struct_df.groupby(['molecule_name', 'atom']).count()
    atoms_per_mol_map = atoms_per_mol_df['atom_index'].unstack().fillna(0)
    atoms_per_mol_map = atoms_per_mol_map.astype(int).to_dict()
    df['num_atoms'] = 0
    for a in atoms_per_mol_map:
        df[f'num_{a}_atoms'] = df['molecule_name'].map(atoms_per_mol_map[a])
        df['num_atoms'] += df[f'num_{a}_atoms']
    return df

train_df = add_dist(train_df, structures_df)
test_df = add_dist(test_df, structures_df)
train_df = add_atom_counts(train_df, structures_df)
test_df = add_atom_counts(test_df, structures_df)
train_df.drop(columns='id', inplace=True)
test_df.drop(columns='id', inplace=True)
train_df['type'] = train_df['type'].map(TYPES_MAP)
test_df['type'] = test_df['type'].map(TYPES_MAP)
train_df = pd.concat((train_df, pd.get_dummies(train_df['type'], prefix='type')), axis=1)
test_df = pd.concat((test_df, pd.get_dummies(test_df['type'], prefix='type')), axis=1)

In [75]:
## Create molecules
def mol_from_xyz(filepath, add_hs=True, compute_dist_centre=False):
    """Wrapper function for calling xyz2mol function."""
    charged_fragments = True  # alternatively radicals are made

    # quick is faster for large systems but requires networkx
    # if you don't want to install networkx set quick=False and
    # uncomment 'import networkx as nx' at the top of the file
    quick = True

    atomicNumList, charge, xyz_coordinates = read_xyz_file(filepath)
    mol, dMat = xyz2mol(atomicNumList, charge, xyz_coordinates,
                        charged_fragments, quick, check_chiral_stereo=False)

    # Compute distance from centroid
    xyz_arr = np.array(xyz_coordinates)
    if compute_dist_centre:
        centroid = xyz_arr.mean(axis=0)
        dFromCentroid = norm(xyz_arr - centroid, axis=1)
    else:
        dFromCentroid = None

    return mol, xyz_arr, dMat, dFromCentroid

# get xyx files and number of molecules
xyz_filepath_list = list(glob(DATA_PATH+'structures/*.xyz'))
xyz_filepath_list.sort()
n_mols = len(xyz_filepath_list)
print('total xyz files # ', n_mols)

# transform .xyz to .mol files and store distance matrices
dist_matrices, mols, mol_ids = {}, {}, {}
for i in tqdm_notebook(range(n_mols)):
    filepath = xyz_filepath_list[i]
    mol_name = filepath.split('/')[-1][:-4]
    mol, _, dist_matrix, _ = mol_from_xyz(filepath)
    mols[mol_name] = mol
    dist_matrices[mol_name] = dist_matrix
    mol_ids[mol_name] = i

total xyz files #  130775


Sanitization error: Explicit valence for atom # 4 C greater than permitted
Sanitization error: Explicit valence for atom # 4 C greater than permitted
Sanitization error: Explicit valence for atom # 5 C greater than permitted
Sanitization error: Explicit valence for atom # 3 C greater than permitted
Sanitization error: Explicit valence for atom # 3 C greater than permitted
Sanitization error: Explicit valence for atom # 5 C greater than permitted
Sanitization error: Explicit valence for atom # 5 C greater than permitted
Sanitization error: Explicit valence for atom # 2 C greater than permitted
Sanitization error: Explicit valence for atom # 7 C greater than permitted
Sanitization error: Explicit valence for atom # 2 C greater than permitted
Sanitization error: Explicit valence for atom # 4 C greater than permitted
Sanitization error: Explicit valence for atom # 3 C greater than permitted
Sanitization error: Explicit valence for atom # 5 C greater than permitted
Sanitization error: Expli

In [76]:
## Store processed dfs
# add molecule ids to dataframes
train_df['molecule_id'] = train_df['molecule_name'].map(mol_ids)
test_df['molecule_id'] = test_df['molecule_name'].map(mol_ids)
train_df.drop(columns=['molecule_name'], inplace=True)
test_df.drop(columns=['molecule_name'], inplace=True)

In [102]:
train_df.to_csv(PATH + 'train_proc_df.csv')
test_df.to_csv(PATH + 'test_proc_df.csv')

In [77]:
## Engineer features

# functions partially sourced from:
# https://deepchem.io/docs/_modules/deepchem/feat/graph_features.html
def one_hot_encoding(x, set):
    one_hot = [int(x == s) for s in set]
    if 0:
        if sum(one_hot)==0: print('one_hot_encoding() return NULL!', x, set)
    return one_hot

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception(f"input {x} not in allowable set{allowable_set}:")
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def get_edge_features(mol, eucl_dist):
    """
    Compute the following features for each bond in 'mol':
        - bond type: categorical {1: single, 2: double, 3: triple,
            4: aromatic} (one-hot)
        - is conjugated: bool {0, 1}
        - is in ring: bool {0, 1}
        - euclidean distance: float
        - normalized eucl distance: float
    """
    n_edges = mol.GetNumBonds()
    features = np.zeros((n_edges, N_EDGE_FEATURES))
    pairs_idx = np.zeros((n_edges, 2))
    for n, e in enumerate(mol.GetBonds()):
        i = e.GetBeginAtomIdx()
        j = e.GetEndAtomIdx()
        dc_e_feats = dc.feat.graph_features.bond_features(e).astype(int)
        features[n, :6] = dc_e_feats
        features[n, 6] = eucl_dist[i, j]
        pairs_idx[n] = i, j
    sorted_idx = pairs_idx[:,0].argsort()
    dists = features[:, 6]
    features[:, 7] = (dists - dists.mean()) / dists.std()
    return features[sorted_idx], pairs_idx[sorted_idx]

def get_atom_features(mol):
    """
    Compute the following features for each atom in 'mol':
        - atom type: H, C, N, O, F (one-hot)
        - degree: 1, 2, 3, 4, 5 (one-hot)
        - Hybridization: SP, SP2, SP3, UNSPECIFIED (one-hot)
        - is aromatic: bool {0, 1}
        - formal charge: int
        - atomic number: float
        - atomic radius: float
        - electro negativity: float
        - donor: bool {0, 1}
        - acceptor: bool {0, 1}
    """
    n_atoms = mol.GetNumAtoms()
    features = np.zeros((n_atoms, N_ATOM_FEATURES))
    for a in mol.GetAtoms():
        sym = a.GetSymbol()
        a_feats = one_hot_encoding(sym, SYMBOLS) \
            + one_hot_encoding(a.GetDegree(), DEGREES) \
            + one_hot_encoding(a.GetHybridization(), HYBRIDIZATIONS) \
            + [a.GetIsAromatic(), a.GetFormalCharge(), a.GetAtomicNum() / 10,
               ATOMIC_RADIUS[sym], ELECTRO_NEG[sym]]
        features[a.GetIdx(), :len(a_feats)] = np.array(a_feats)

    feat_factory = ChemicalFeatures.BuildFeatureFactory(FDEF)
    try:
        chem_feats = feat_factory.GetFeaturesForMol(mol)
        for t in range(len(chem_feats)):
            if chem_feats[t].GetFamily() == 'Donor':
                for i in chem_feats[t].GetAtomIds():
                    features[i, -2] = 1
            elif chem_feats[t].GetFamily() == 'Acceptor':
                for i in chem_feats[t].GetAtomIds():
                    features[i, -1] = 1
    except RuntimeError as e:
        print(e)

    return features

In [78]:
# create features
atom_features = []
edge_features = []
pairs_idx = []
atom_to_m_id = []
edge_to_m_id = []

for i, m_name in enumerate(mols):
    print_progress(i)
    m_id, mol, = mol_ids[m_name], mols[m_name]
    dist_matrix = dist_matrices[m_name]
    n_atoms, n_edges = mol.GetNumAtoms(), mol.GetNumBonds()
    
    atom_features.append(get_atom_features(mol))

    e_feats, p_idx = get_edge_features(mol, dist_matrix)
    edge_features.append(e_feats)
    pairs_idx.append(p_idx)
    
    atom_to_m_id.append(np.repeat(m_id, n_atoms))
    edge_to_m_id.append(np.repeat(m_id, n_edges))

0
10000
Pre-condition Violation
	getNumImplicitHs() called without preceding call to calcImplicitValence()
	Violation occurred on line 152 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_implicitValence > -1
	RDKIT: 2019.03.2
	BOOST: 1_68

20000
30000
Pre-condition Violation
	getExplicitValence() called without call to calcExplicitValence()
	Violation occurred on line 161 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_explicitValence > -1
	RDKIT: 2019.03.2
	BOOST: 1_68

Pre-condition Violation
	getNumImplicitHs() called without preceding call to calcImplicitValence()
	Violation occurred on line 152 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_implicitValence > -1
	RDKIT: 2019.03.2
	BOOST: 1_68

40000
Pre-condition Violation
	getExplicitValence() called without call to calcExplicitValence()
	Violation occurred on line 161 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_explicitValence > -1
	RDKIT: 2019.03.2
	BOOST: 1_68

Pre-condition Violation
	getNumImplicitHs(

In [82]:
atom_features = pd.DataFrame(np.concatenate(atom_features), columns=ATOM_COLUMNS)
edge_features = pd.DataFrame(np.concatenate(edge_features), columns=EDGE_COLUMNS)
pairs_idx = np.concatenate(pairs_idx)
edge_features['idx_0'] = pairs_idx[:,0]
edge_features['idx_1'] = pairs_idx[:,1]
atom_features['molecule_id'] = np.concatenate(atom_to_m_id) 
edge_features['molecule_id'] = np.concatenate(edge_to_m_id) 

In [95]:
atom_features.to_csv(PATH + 'atom_df.csv')
edge_features.to_csv(PATH + 'edge_df.csv')